In [ ]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Simple notebook pipeline 

Welcome to your first steps with Kubeflow Pipelines (KFP). In this notebook, we will demo: 

* Defining a Kubeflow pipeline with the KFP SDK
* Creating an experiment and submitting pipelines to the KFP run time environment using the KFP SDK 

Reference documentation: 
* https://www.kubeflow.org/docs/pipelines/sdk/sdk-overview/
* https://www.kubeflow.org/docs/pipelines/sdk/build-component/

### Prerequisites: Install the pipelines SDK
If you followed the [notebook setup instructions](https://www.kubeflow.org/docs/notebooks/setup/) then you don't have to run the next cell.  You only need to install the KFP package once in your notebook server.  You can find the latest KFP package [here](https://github.com/kubeflow/pipelines/releases).

In [ ]:
# If you've already done his during setup you don't need to do it again
KFP_PACKAGE = 'https://storage.googleapis.com/ml-pipeline/release/0.1.23/kfp.tar.gz'
!pip3 install $KFP_PACKAGE --upgrade

### Setup
<b> Important: </b> Make sure to use your GCP project name and a GCS bucket for saving the pipelines.  If you haven't already, [setup a GCS bucket](https://cloud.google.com/storage/docs/creating-buckets).

In [ ]:
# Set your output directory and project name. 
PROJECT_NAME =  '[YOUR-GCP-PROJECT-NAME]'           # 'Your-Gcp-Project-Name'
OUTPUT_DIR = 'gs://[YOUR-GCS-BUCKET]/assets'        # A path for asset outputs

In [ ]:
EXPERIMENT_NAME = 'Simple notebook pipeline'                    # Name of the experiment in the UI
BASE_IMAGE = 'tensorflow/tensorflow:2.0.0b0-py3'    # Base image used for components in the pipeline

In [ ]:
import kfp
import kfp.dsl as dsl
from kfp import compiler

### Create pipeline component

#### Create python function

In [ ]:
@dsl.python_component(
    name='add_op',
    description='adds two numbers',
    base_image=BASE_IMAGE  # you can define the base image here, or when you build in the next step. 
)
def add(a: float, b: float) -> float:
    '''Calculates sum of two arguments'''
    print(a, '+', b, '=', a + b)
    return a + b

#### Build pipeline component from the function

In [ ]:
# Docker image for the pipeline component
TARGET_IMAGE = 'gcr.io/%s/add-op:latest' % PROJECT_NAME 

# The return value "DeployerOp" represents a step that is used in a pipeline
add_op = compiler.build_python_component(
    component_func=add,
    staging_gcs_path=OUTPUT_DIR,
    base_image=BASE_IMAGE,
    target_image=TARGET_IMAGE)

### Build a pipeline using the component

In [ ]:
@dsl.pipeline(
   name='Calculation pipeline',
   description='A toy pipeline that performs arithmetic calculations.'
)
def calc_pipeline(
   a='0',
   b='7',
   c='17',
):
    #Passing pipeline parameter and a constant value as operation arguments
    add_task = add_op(a, 4) #Returns a dsl.ContainerOp class instance. 
    
    #You can create explicit dependency between the tasks using xyz_task.after(abc_task)
    add_2_task = add_op(a, b)
    
    add_3_task = add_op(add_task.output, add_2_task.output)

### Compile and run the pipeline

In [ ]:
pipeline_func = calc_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
compiler.Compiler().compile(pipeline_func, pipeline_filename)

#### Create an Experiment in the Pipeline System

Pipeline system requires an "Experiment" to group pipeline runs. You can create a new experiment, or call client.list_experiments() to get existing ones.

In [ ]:
#Get or create an experiment and submit a pipeline run
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

#### Submit the pipeline for execution

In [ ]:
#Specify pipeline argument values
arguments = {'a': '7', 'b': '8'}

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

#This link leads to the run information page. 
#Note: There is a bug in JupyterLab that modifies the URL and makes the link stop working

### That's it!
You just created and deployed your first pipeline in Kubeflow!  You can put more complex python code within the functions, and you can import any libraries that are included in the base image (you can use [VersionedDependencies](https://kubeflow-pipelines.readthedocs.io/en/latest/source/kfp.compiler.html#kfp.compiler.VersionedDependency) to import libraries not included in the base image). 